In [ ]:
!pip install gospl -U --no-deps

In [1]:
import shutil
import os
for dirpath, dirname, files in os.walk("/live"):
    if "inputparser.py" in files:
        infile = os.path.join(dirpath,"inputparser.py")
        break
print(infile)
shutil.copyfile(
    infile,
    "/usr/local/lib/python3.7/dist-packages/gospl/tools/inputparser.py")

/live/lib/test/inputparser.py


'/usr/local/lib/python3.7/dist-packages/gospl/tools/inputparser.py'

In [1]:
#import all necessary packages

import os
import meshio
import meshplex
import jigsawpy
import numpy as np
import pyvista as pv
from netCDF4 import Dataset
from scipy import interpolate
from scipy import ndimage
from time import process_time
from gospl._fortran import definegtin
import mapOutputs as mout


In [2]:
#input parameters

#start time (Ma)
st_time = 0

#input files
#elevation (.nc, with elevation in m)
elev_map = "wegs.nc"
#precipitation (.nc, with average annual precipitation in m/s)
precip_map = [
#    'tc_pr_basic_ann_av.nc'
]
#uplift
uplift_map = [
#    'gos_up.nc'
]

#resolution of mesh (km):
#deep ocean
res_ocean = 1000
#continental shelf
res_shelf = 10
#land
res_land = 5
#division between resolution levels (elevation, m):
#deep ocean/shelf
depth_ocean = -1000
#shelf/land
depth_shelf = -200

#planet radius in km
planrad = 6130


In [3]:
#Create input folder for gospl

input_path = "input_folder"
if not os.path.exists(input_path):
    os.makedirs(input_path)

In [ ]:
#Function to construct topography mesh based on elevation file

def buildRegMesh(infile, outfile, key):
    t0 = process_time()
    
    data = Dataset(infile, "r", format="NETCDF4")
    img = np.fliplr(data[key][:, :].T)
    width = img.shape[0]
    height = img.shape[1]
    
    lon = np.linspace(-180.0, 180, width)
    lat = np.linspace(-90.0, 90, height)
    print("Read map (%0.02f seconds)" % (process_time() - t0))
    
    value = np.round(img.flatten(), 3)
    to = process_time()
    f = open(outfile, "w+")
    f.write("mshid=3;ellipsoid-grid\n")
    f.write("mdims=2\n")
    
    f.write("coord=1;%d\n" % (len(lon)))
    for k in range(len(lon)):
        f.write(str(lon[k]) + "\n")

    f.write("coord=2;%d\n" % (len(lat)))
    for k in range(len(lat)):
        f.write(str(lat[k]) + "\n")

    f.write("value=%d;1\n" % (len(value)))
    for k in range(len(value)):
        f.write(str(value[k]) + "\n")
    
    f.close()
    
    print("Wrote topo mesh file (%0.02f seconds)" % (process_time() - t0))
    
    return

In [ ]:
#Run that function with your topography input (will take a couple minutes)

topofile = os.path.join(input_path, "topo.msh")
buildRegMesh(elev_map, topofile, "z")

In [ ]:
#Function to produce a global unstructured mesh with varying resolution scale based on elevation

def getInitialMesh(topofile, meshfile, spacefile, outfile, dst_path, hfn, planrad=6.371e003):

    t0 = process_time()
    opts = jigsawpy.jigsaw_jig_t()
    topo = jigsawpy.jigsaw_msh_t()
    geom = jigsawpy.jigsaw_msh_t()
    mesh = jigsawpy.jigsaw_msh_t()
    hmat = jigsawpy.jigsaw_msh_t()

    jigsawpy.loadmsh(topofile, topo)
    print("Loaded topo grid (%0.02f seconds)" % (process_time() - t0))

    t0 = process_time()
    opts.geom_file = os.path.join(dst_path, "topology.msh")
    opts.jcfg_file = os.path.join(dst_path, "config.jig")
    opts.mesh_file = meshfile
    opts.hfun_file = spacefile

    geom.mshID = "ellipsoid-mesh"
    geom.radii = np.full(3, planrad, dtype=geom.REALS_t)
    jigsawpy.savemsh(opts.geom_file, geom)

    hmat.mshID = "ellipsoid-grid"
    hmat.radii = geom.radii
    hmat.xgrid = topo.xgrid * np.pi / 180.0
    hmat.ygrid = topo.ygrid * np.pi / 180.0

    # Set HFUN gradient-limiter
    hmat.value = np.full(topo.value.shape, hfn[0], dtype=hmat.REALS_t)
    hmat.value[topo.value > hfn[3]] = hfn[1]
    hmat.value[topo.value > hfn[4]] = hfn[2]

    hmat.slope = np.full(topo.value.shape, +0.050, dtype=hmat.REALS_t)
    jigsawpy.savemsh(opts.hfun_file, hmat)
    jigsawpy.cmd.marche(opts, hmat)
    print("Built space function (%0.02f seconds)" % (process_time() - t0))

    t0 = process_time()
    opts.hfun_scal = "absolute"
    opts.hfun_hmax = float("inf")  # null HFUN limits
    opts.hfun_hmin = float(+0.00)

    opts.mesh_dims = +2  # 2-dim. simplexes

    opts.optm_qlim = +9.5e-01  # tighter opt. tol
    opts.optm_iter = +32
    opts.optm_qtol = +1.0e-05

    jigsawpy.cmd.tetris(opts, 3, mesh)
    print("Performed triangulation (%0.02f seconds)" % (process_time() - t0))

    t0 = process_time()
    apos = jigsawpy.R3toS2(geom.radii, mesh.point["coord"][:])

    apos = apos * 180.0 / np.pi

    zfun = interpolate.RectBivariateSpline(topo.ygrid, topo.xgrid, topo.value)

    mesh.value = zfun(apos[:, 1], apos[:, 0], grid=False)

    jigsawpy.savevtk(outfile, mesh)
    jigsawpy.savemsh(opts.mesh_file, mesh)
    print("Got unstructured mesh (%0.02f seconds)" % (process_time() - t0))

    return

In [ ]:
#Run that function with your resolution inputs (will take a while, possibly over an hour)

meshfile = os.path.join(input_path, "mesh" + str(st_time) + ".msh")
spacefile = os.path.join(input_path, "spac" + str(st_time) + ".msh")
outfile = os.path.join(input_path, "mesh" + str(st_time) + ".vtk")
hfn = np.zeros(5)
hfn[0] = res_ocean
hfn[1] = res_shelf
hfn[2] = res_land
hfn[3] = depth_ocean
hfn[4] = depth_shelf
getInitialMesh(topofile, meshfile, spacefile, outfile, input_path, hfn, planrad)

In [ ]:
#Prepare some variables for the next steps

umesh = meshio.read(outfile)
coords = umesh.points
coords = coords * 1000
cells = umesh.cells_dict["triangle"]
elev = umesh.point_data["value"]

In [ ]:
#converts from mesh coordinates to lon/lat

def xyz2lonlat(coords, radius=planrad*1000):

    gLonLat = np.zeros((len(coords), 2))

    gLonLat[:, 1] = -np.arcsin(coords[:, 2] / radius)
    gLonLat[:, 0] = np.arctan2(coords[:, 1], coords[:, 0])
    gLonLat[:, 1] = np.mod(np.degrees(gLonLat[:, 1]) + 90, 180.0)
    gLonLat[:, 0] = np.mod(np.degrees(gLonLat[:, 0]) + 180.0, 360.0)

    return gLonLat

In [ ]:
#Function to prepare elevation input for gospl

def gosplElev(coords, cells, elev, gmesh, visvtk=False):

    Gmesh = meshplex.MeshTri(coords, cells)
    s = Gmesh.idx_hierarchy.shape
    a = np.sort(Gmesh.idx_hierarchy.reshape(s[0], -1).T)

    if meshplex.__version__ >= "0.16.0":
        Gmesh.edges = {"points": np.unique(a, axis=0)}
        ngbNbs, ngbID = definegtin(
            len(coords), Gmesh.cells("points"), Gmesh.edges["points"]
        )
    elif meshplex.__version__ >= "0.14.0":
        Gmesh.edges = {"points": np.unique(a, axis=0)}
        ngbNbs, ngbID = definegtin(
            len(coords), Gmesh.cells["points"], Gmesh.edges["points"]
        )
    else:
        Gmesh.edges = {"nodes": np.unique(a, axis=0)}
        ngbNbs, ngbID = definegtin(
            len(coords), Gmesh.cells["nodes"], Gmesh.edges["nodes"]
        )
    
    if visvtk:
        paleovtk = gmesh + ".vtk"
        vis_mesh = meshio.Mesh(coords, {"triangle": cells}, point_data={"z": elev})
        meshio.write(paleovtk, vis_mesh)
        print("Writing VTK file {}".format(paleovtk))

    np.savez_compressed(gmesh, v=coords, c=cells, n=ngbID[:, :8].astype(int), z=elev)

    return

In [ ]:
#Runs that function

npzelev = os.path.join(input_path, str(st_time) + "Ma")
gosplElev(coords, cells, elev, npzelev, visvtk=False)

In [ ]:
#Function to prepare precipitation input for gospl

def gosplRain(coords, cells, paleorain, rainmesh, visvtk=False, filter=2):

    lonlat = xyz2lonlat(coords, radius=planrad*1000)

    data = Dataset(paleorain, "r", format="NETCDF4")
    try: 
        paleorain = data["pr"][0,:, :].T
    except:
        paleorain = data["z"][:, :].T

    # Map mesh coordinates
    ilons = paleorain.shape[0] * lonlat[:, 0] / float(paleorain.shape[0])
    ilats = paleorain.shape[1] * lonlat[:, 1] / float(paleorain.shape[1])

    icoords = np.stack((ilons, ilats))
    paleorain = ndimage.gaussian_filter(paleorain, sigma=filter)

    rlons = icoords[0, :] * paleorain.shape[0] / 360.0
    rlats = icoords[1, :] * paleorain.shape[1] / 180.0

    rcoords = np.zeros(icoords.shape)
    rcoords[0, :] = rlons
    rcoords[1, :] = rlats

    # Interpolate the paleogrid on global mesh
    meshd = ndimage.map_coordinates(paleorain, rcoords, order=2, mode="nearest").astype(
        np.float64,
    )

    # Conversion from mm/day to m/yr
    #meshd *= 365.2422 / 1000.0
    
    #m/s to m/year
    meshd *= 365.25 * 24 * 60 * 60

    # Save the mesh as compressed numpy file for global simulation
    np.savez_compressed(rainmesh, r=meshd)

    if visvtk:
        paleovtk = rainmesh + ".vtk"
        vis_mesh = meshio.Mesh(coords, {"triangle": cells}, point_data={"r": meshd})
        meshio.write(paleovtk, vis_mesh)
        print("Writing VTK file {}".format(paleovtk))

    return meshd

In [ ]:
#Runs that function for each precipitation file and names them in order

i=1
for rain in precip_map:
    npzrain = os.path.join(input_path, "rain"+str(i))
    meshd = gosplRain(coords, cells, rain, npzrain, visvtk=False, filter=2)
    i+=1

In [ ]:
#Function to prepare uplift input for gospl

def gosplUplift(coords, cells, paleouplift, upliftmesh, visvtk=False, filter=2):

    lonlat = xyz2lonlat(coords, radius=planrad*1000)

    data = Dataset(paleouplift, "r", format="NETCDF4")
    paleouplift = data["z"][:, :].T

    # Map mesh coordinates
    ilons = paleouplift.shape[0] * lonlat[:, 0] / float(paleouplift.shape[0])
    ilats = paleouplift.shape[1] * lonlat[:, 1] / float(paleouplift.shape[1])

    icoords = np.stack((ilons, ilats))
    paleouplift = ndimage.gaussian_filter(paleouplift, sigma=filter)

    rlons = icoords[0, :] * paleouplift.shape[0] / 360.0
    rlats = icoords[1, :] * paleouplift.shape[1] / 180.0

    rcoords = np.zeros(icoords.shape)
    rcoords[0, :] = rlons
    rcoords[1, :] = rlats

    # Interpolate the paleogrid on global mesh
    meshd = ndimage.map_coordinates(paleouplift, rcoords, order=2, mode="nearest").astype(
        np.float64,
    )
    
    meshd /= 1000

    # Save the mesh as compressed numpy file for global simulation
    np.savez_compressed(upliftmesh, z=meshd)

    if visvtk:
        paleovtk = upliftmesh + ".vtk"
        vis_mesh = meshio.Mesh(coords, {"triangle": cells}, point_data={"r": meshd})
        meshio.write(paleovtk, vis_mesh)
        print("Writing VTK file {}".format(paleovtk))

    return meshd

In [ ]:
#Runs that function for each uplift file and names them in order

i=1
for up in uplift_map:
    npzuplift = os.path.join(input_path, "uplift"+str(i))
    meshu = gosplUplift(coords, cells, up, npzuplift, visvtk=False, filter=2)
    i+=1

In [ ]:
#Displays mesh data on an interactable globe
#If you don't see one, make sure the script is "Trusted" in the upper right and/or reload the page 
npzelev = os.path.join(input_path, str(st_time) + "Ma")
paleovtk = npzelev + ".vtk"

# Define mesh
if len(uplift_map) < 1 and len(precip_map) < 1:
    vis_mesh = meshio.Mesh(coords, {"triangle": cells}, 
                       point_data={"elev": elev})
elif len(uplift_map) < 1:
    vis_mesh = meshio.Mesh(coords, {"triangle": cells}, 
                       point_data={"elev": elev, "precip": meshd})
elif len(precip_map) < 1:
    vis_mesh = meshio.Mesh(coords, {"triangle": cells}, 
                       point_data={"elev": elev, "uplift": meshu})
else:
    vis_mesh = meshio.Mesh(coords, {"triangle": cells}, 
                       point_data={"elev": elev, "precip": meshd, "uplift": meshu})

# Write it to disk
meshio.write(paleovtk, vis_mesh)

print("Writing VTK input file as {}".format(paleovtk))

mesh = pv.read(paleovtk)
melev = mesh.get_array(name='elev')

planRadius = planrad*1000
scale = 20.
factor = 1.+ (melev/planRadius)*scale

mesh.points[:, 0] *= factor
mesh.points[:, 1] *= factor
mesh.points[:, 2] *= factor

contour = mesh.contour([0])

plotter = pv.PlotterITK()
plotter.add_mesh(mesh, scalars="elev")
plotter.add_mesh(contour, color="black", opacity=1.)

plotter.show()

In [4]:
# Runs model; can take hours but it'll report its progress as it runs

#multi-core; replace number with your available cpu cores
!mpirun -np 8 python3 runModel.py -i input.yml

#single-core
# %run script/runModel.py -i input.yml

--- Initialisation Phase (44.19 seconds)
+++ Output Simulation Time: 0.00 years
--- Computational Step                       (164.96 seconds)
--- Computational Step                       (162.41 seconds)
--- Computational Step                       (165.75 seconds)
--- Computational Step                       (164.86 seconds)
--- Computational Step                       (153.76 seconds)
LinearSolver failed to converge after %d iterations 0
with reason: %s DIVERGED_PCSETUP_FAILED
input vector:  0.0 1266059562492.4067
--- Computational Step                       (148.99 seconds)
LinearSolver failed to converge after %d iterations 0
with reason: %s DIVERGED_PCSETUP_FAILED
input vector:  0.0 1276366361310.602
--- Computational Step                       (149.18 seconds)
LinearSolver failed to converge after %d iterations 0
with reason: %s DIVERGED_PCSETUP_FAILED
input vector:  0.0 1303986308734.421
--- Computational Step                       (156.82 seconds)
--- Computational Step        

In [7]:
#Displays output on interactable globe

stp = 5
uplift_forcing = False

# Reading the final output generated by gospl
output = mout.mapOutputs(path='./', filename='input.yml',step=stp, uplift=False)
output.exportVTK(f'step{stp}.vtk')

mesh = pv.read(f'step{stp}.vtk')
elev = mesh.get_array(name='elev')

planRadius = planrad*1000
scale = 20.
factor = 1.+ (elev/planRadius)*scale

mesh.points[:, 0] *= factor
mesh.points[:, 1] *= factor
mesh.points[:, 2] *= factor

contour = mesh.contour([0])

plot = pv.PlotterITK()
plot.add_mesh(mesh, scalars="elev")
plot.add_mesh(contour, color="black", opacity=1.)
plot.show()

Writing VTK file step5.vtk


Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [8]:
#Saves output to netcdf file
import mapOutputs as mout

stp = 5
reso = 0.06

grid = mout.mapOutputs(path='./', filename='input.yml',step=stp, uplift=False)
grid.getData(stp)
grid.buildLonLatMesh(res=reso, nghb=3)
grid.exportNetCDF(ncfile = 'out.nc')


In [9]:
#Trimmed Version; saves only elevation, in 4 quadrants
import mapOutputs as mout

stp = 5
reso = 0.02

grid = mout.mapOutputs(path='./', filename='input.yml',step=stp, uplift=False)
grid.getData_e(stp)
grid.buildLonLatMesh_e(res=reso, nghb=3, box = [-180.0,0.0,0.0,90.0])
grid.exportNetCDF_e(ncfile = 'outnw.nc')
grid = mout.mapOutputs(path='./', filename='input.yml',step=stp, uplift=False)
grid.getData_e(stp)
grid.buildLonLatMesh_e(res=reso, nghb=3, box = [0.0,0.0,180.0,90.0])
grid.exportNetCDF_e(ncfile = 'outne.nc')
grid = mout.mapOutputs(path='./', filename='input.yml',step=stp, uplift=False)
grid.getData_e(stp)
grid.buildLonLatMesh_e(res=reso, nghb=3, box = [-180.0,-90.0,0.0,0.0])
grid.exportNetCDF_e(ncfile = 'outsw.nc')
grid = mout.mapOutputs(path='./', filename='input.yml',step=stp, uplift=False)
grid.getData_e(stp)
grid.buildLonLatMesh_e(res=reso, nghb=3, box = [0.0,-90.0,180.0,0.0])
grid.exportNetCDF_e(ncfile = 'outse.nc')


In [ ]:
#saves only elevation, in 16 pieces
import mapOutputs as mout

stp = 3
reso = 0.02

grid = mout.mapOutputs(path='./', filename='input.yml',step=stp, uplift=False)
grid.getData_e(stp)
grid.buildLonLatMesh_e(res=reso, nghb=3, box = [-180.0,45.0,-90.0,90.0])
grid.exportNetCDF_e(ncfile = 'outnnww.nc')
grid.buildLonLatMesh_e(res=reso, nghb=3, box = [-90.0,45.0,0.0,90.0])
grid.exportNetCDF_e(ncfile = 'outnnw.nc')
grid.buildLonLatMesh_e(res=reso, nghb=3, box = [0.0,45.0,90.0,90.0])
grid.exportNetCDF_e(ncfile = 'outnne.nc')
grid.buildLonLatMesh_e(res=reso, nghb=3, box = [90.0,45.0,180.0,90.0])
grid.exportNetCDF_e(ncfile = 'outnnee.nc')
grid.buildLonLatMesh_e(res=reso, nghb=3, box = [-180.0,0.0,-90.0,45.0])
grid.exportNetCDF_e(ncfile = 'outnww.nc')
grid.buildLonLatMesh_e(res=reso, nghb=3, box = [-90.0,0.0,0.0,45.0])
grid.exportNetCDF_e(ncfile = 'outnw.nc')
grid.buildLonLatMesh_e(res=reso, nghb=3, box = [0.0,0.0,90.0,45.0])
grid.exportNetCDF_e(ncfile = 'outne.nc')
grid.buildLonLatMesh_e(res=reso, nghb=3, box = [90.0,0.0,180.0,45.0])
grid.exportNetCDF_e(ncfile = 'outnee.nc')
grid.buildLonLatMesh_e(res=reso, nghb=3, box = [-180.0,-45.0,-90.0,0.0])
grid.exportNetCDF_e(ncfile = 'outsww.nc')
grid.buildLonLatMesh_e(res=reso, nghb=3, box = [-90.0,-45.0,0.0,0.0])
grid.exportNetCDF_e(ncfile = 'outsw.nc')
grid.buildLonLatMesh_e(res=reso, nghb=3, box = [0.0,-45.0,90.0,0.0])
grid.exportNetCDF_e(ncfile = 'outse.nc')
grid.buildLonLatMesh_e(res=reso, nghb=3, box = [90.0,-45.0,180.0,0.0])
grid.exportNetCDF_e(ncfile = 'outsee.nc')
grid.buildLonLatMesh_e(res=reso, nghb=3, box = [-180.0,-90.0,-90.0,-45.0])
grid.exportNetCDF_e(ncfile = 'outssww.nc')
grid.buildLonLatMesh_e(res=reso, nghb=3, box = [-90.0,-90.0,0.0,-45.0])
grid.exportNetCDF_e(ncfile = 'outssw.nc')
grid.buildLonLatMesh_e(res=reso, nghb=3, box = [0.0,-90.0,90.0,-45.0])
grid.exportNetCDF_e(ncfile = 'outsse.nc')
grid.buildLonLatMesh_e(res=reso, nghb=3, box = [90.0,-90.0,180.0,-45.0])
grid.exportNetCDF_e(ncfile = 'outssee.nc')